In [62]:
import pandas as pd 
import numpy as np 

In [63]:
s3_path = (
    "s3a://pop-eu-prod/390/200_feed/0/0/4e/056920.txt" 
)

details_df = pd.read_csv(s3_path, sep="\t")
details_df

,SKU,Stock,Price,WholeSale Price,Sin
0,720-021,In Stock,99.99 GBP,73.99 GBP,1O366AQUOA000
1,841-094,In Stock,179.99 GBP,119.99 GBP,15K2C8K9AF801
2,AA1-069RD,Out of Stock,59.99 GBP,44.99 GBP,15KFM0J8AF001
3,835-388WT,In Stock,69.99 GBP,49.99 GBP,15KGDBUMIF001
4,845-171,8,339.99 GBP,280.99 GBP,15KDIN5LHH801
...,...,...,...,...,...
10806,833-695V72DB,19,179.99 GBP,164.99 GBP,21D6BGADHKO01
10807,833-695V74YL,In Stock,179.99 GBP,145.99 GBP,21D6BGAS1TO00
10808,833-695V72LG,In Stock,459.99 GBP,139.99 GBP,21DQV4CKHNO00
10809,833-695V74BG,Out of Stock,179.99 GBP,154.99 GBP,21FOO8TFPL801


In [64]:
def numerate_stock(stock):
    if stock == 'Out of Stock':
        return 0
    elif stock == 'In Stock':
        return 30
    else:
        try:
            return int(stock)
        except Exception:
            return None

numerate_stock_df = details_df.copy()
numerate_stock_df['Stock'] = details_df['Stock'].apply(numerate_stock)
numerate_stock_df.rename(columns={'SKU': 'Custom label (SKU)'}, inplace=True)
numerate_stock_df


,Custom label (SKU),Stock,Price,WholeSale Price,Sin
0,720-021,30,99.99 GBP,73.99 GBP,1O366AQUOA000
1,841-094,30,179.99 GBP,119.99 GBP,15K2C8K9AF801
2,AA1-069RD,0,59.99 GBP,44.99 GBP,15KFM0J8AF001
3,835-388WT,30,69.99 GBP,49.99 GBP,15KGDBUMIF001
4,845-171,8,339.99 GBP,280.99 GBP,15KDIN5LHH801
...,...,...,...,...,...
10806,833-695V72DB,19,179.99 GBP,164.99 GBP,21D6BGADHKO01
10807,833-695V74YL,30,179.99 GBP,145.99 GBP,21D6BGAS1TO00
10808,833-695V72LG,30,459.99 GBP,139.99 GBP,21DQV4CKHNO00
10809,833-695V74BG,0,179.99 GBP,154.99 GBP,21FOO8TFPL801


In [65]:
file_path = "Data/eBay-edit-price-quantity-template-2026-01-05-11278396261.csv"

metadata_row = pd.read_csv(file_path, nrows=1)
ebay_listings = pd.read_csv(file_path, skiprows=1)

ebay_listings.head()


,Action,Category name,Item number,Title,Listing site,Currency,Start price,Buy It Now price,Available quantity,Relationship,Relationship details,Custom label (SKU)
0,Revise,Mirrors (20580),406140405400,Full Length Mirror Floor Standing or Wall Moun...,UK,GBP,64.99,NaN,10,NaN,NaN,831-387
1,Revise,Strollers (116380),406140405488,Pet Stroller Foldable Travel Carriage w/ Brake...,UK,GBP,48.99,NaN,10,NaN,NaN,D00-127GY
2,Revise,Fences & Pens (20748),406140405499,Pressure Fitted Pet Dog Safety Gate Metal Fenc...,UK,GBP,40.99,NaN,10,NaN,NaN,D06-058
3,Revise,Gazebos & Pergolas (180995),406140405512,3(m) 2 Tier Garden Gazebo Top Cover Replacemen...,UK,GBP,29.99,NaN,10,NaN,NaN,84C-041GY
4,Revise,Bathroom Cabinets (176991),406140405516,60x60cm Under-Sink Storage Cabinet w/ Adjustab...,UK,GBP,55.99,NaN,10,NaN,NaN,834-279


In [66]:
# --- 1. PREPARE THE DATA ---

# Assuming 'numerate_stock_df' and 'ebay_listings' are already Pandas DataFrames
# Select necessary columns from stock update
stock_subset = numerate_stock_df[['Custom label (SKU)', 'Stock', 'WholeSale Price']]

# Join the data: Equivalent to Spark "left join"
# We merge on SKU, then drop the old quantity/price columns
ebay_listings_updated = ebay_listings.merge(
    stock_subset, 
    on='Custom label (SKU)', 
    how='left'
)

# Drop the old columns and rename the new ones to take their place
ebay_listings_updated = ebay_listings_updated.drop(columns=['Available quantity', 'Start price'])
ebay_listings_updated = ebay_listings_updated.rename(columns={
    'Stock': 'Available quantity',
    'WholeSale Price': 'Start price'
})

# --- 2. REORDER COLUMNS ---

# Get the list of current columns
cols = list(ebay_listings.columns)

# If you need to ensure 'Available quantity' and 'Start price' are in specific spots:
# We find the index of SKU and insert the updated columns right after it
sku_idx = cols.index('Custom label (SKU)')
# Remove them if they exist in the list to avoid duplicates, then re-insert
current_cols = [c for c in ebay_listings_updated.columns if c not in ['Available quantity', 'Start price']]
new_order = (
    current_cols[:sku_idx + 1] + 
    ['Available quantity', 'Start price'] + 
    current_cols[sku_idx + 1:]
)

ebay_listings_updated = ebay_listings_updated[new_order]

# --- 3. CONVERT TO STRINGS & FILL NULLS ---

# Cast everything to string and replace NaN with empty strings
ebay_listings_updated = ebay_listings_updated.astype(str).replace('nan', '')

# --- 4. METADATA STACKING ---

# 1. Capture the actual data column names (e.g., SKU, Title, Price...)
actual_columns = ebay_listings_updated.columns.tolist()

# 2. Get the metadata values as a list
metadata_values = metadata_row.columns.tolist()

# 3. Align the metadata list to the data width
# Pad with empty strings if the data table is wider than the metadata
padding_needed = len(actual_columns) - len(metadata_values)
if padding_needed > 0:
    metadata_values += [""] * padding_needed
else:
    # Trim if the metadata is somehow wider than the data table
    metadata_values = metadata_values[:len(actual_columns)]

# 4. Create the Metadata DataFrame
# IMPORTANT: We force 'columns=actual_columns' so Pandas stacks them vertically
metadata_df_row = pd.DataFrame([metadata_values], columns=actual_columns)

# 5. Create the Header Row (the previous columns as the first row of data)
column_headers_row = pd.DataFrame([actual_columns], columns=actual_columns)

# 6. Stack them: Metadata -> Headers -> Data
ebay_listings_final = pd.concat(
    [metadata_df_row, column_headers_row, ebay_listings_updated], 
    ignore_index=True
)

# --- 5. OUTPUT ---
# Note: When saving to CSV, use header=False because your Row 0 and Row 1 
# are now technically your "headers".
ebay_listings_final.to_csv("Data/ebay_edit_template_upload.csv", header=False, index=False)

In [67]:
ebay_listings_final[ebay_listings_final['Custom label (SKU)'] == 'AA1-056V02BK']

,Action,Category name,Item number,Title,Listing site,Currency,Buy It Now price,Relationship,Relationship details,Custom label (SKU),Available quantity,Start price
50,Revise,Electric Scooters (47349),406140406225,"Foldable Kids Electric Scooter, for Ages 7-14 ...",UK,GBP,,,,AA1-056V02BK,30.0,92.99 GBP


In [68]:
import math
def multiply_price(price):
    try:
        
        new_price = float(price.split(' ')[0])*1.15
        final_price = math.ceil(new_price) - 0.01
        
        return f"{final_price:.2f}"
    except (ValueError, TypeError):
        # If it's still failing, return a safe default or 0.00 to see where it breaks
        return "0.00"

In [69]:

numerate_stock_df['WholeSale Price'] = numerate_stock_df['WholeSale Price'].apply(multiply_price)
numerate_stock_df

,Custom label (SKU),Stock,Price,WholeSale Price,Sin
0,720-021,30,99.99 GBP,85.99,1O366AQUOA000
1,841-094,30,179.99 GBP,137.99,15K2C8K9AF801
2,AA1-069RD,0,59.99 GBP,51.99,15KFM0J8AF001
3,835-388WT,30,69.99 GBP,57.99,15KGDBUMIF001
4,845-171,8,339.99 GBP,323.99,15KDIN5LHH801
...,...,...,...,...,...
10806,833-695V72DB,19,179.99 GBP,189.99,21D6BGADHKO01
10807,833-695V74YL,30,179.99 GBP,167.99,21D6BGAS1TO00
10808,833-695V72LG,30,459.99 GBP,160.99,21DQV4CKHNO00
10809,833-695V74BG,0,179.99 GBP,178.99,21FOO8TFPL801
